In [1]:
pip install opencv-python

PermissionError: [WinError 5] Access is denied

In [3]:
pip install matplotlib

PermissionError: [WinError 5] Access is denied

In [ ]:
pip install numpy

In [ ]:
pip install os

In [ ]:
import cv2
import os
image_file = "data/Cerificate.png"
img = cv2.imread(image_file)

In [ ]:
import numpy as np 
import cv2
img = cv2.imread("data/Certificate.png") 
cv2.imshow("Image",img) # show the image, name of the display

In [ ]:
def display_image_in_actual_size(im_path):

    dpi = 80
    im_data = plt.imread(im_path)
    height, width, depth = im_data.shape

    # What size does the figure need to be in inches to fit the image?
    figsize = width / float(dpi), height / float(dpi)

    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])

    # Hide spines, ticks, etc.
    ax.axis('off')

    # Display the image.
    ax.imshow(im_data, cmap='gray')

    plt.show()

In [ ]:
display(image_file)

In [ ]:
inverted_image = cv2.bitwise_not(img)
cv2.imwrite("temp/inverted.png", inverted_image)

In [ ]:
display('temp/inverted.png')

In [ ]:
def grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [ ]:
gray_image = grayscale(img)
cv2.imwrite("temp/gray.png", gray_image)

In [ ]:
display("temp/gray.png")

In [ ]:
thresh, im_bw = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY)
cv2.imwrite("temp/bw_image.png", im_bw)

In [ ]:
display("temp/bw_image.png")

In [ ]:
def noise_removal(image):
    import numpy as np
    kernal = np.ones((1,1), np.uint8)
    image = cv2.dilate(image, kernal, iterations=1)
    kernal = np.ones((1,1), np.uint8)
    image = cv2.erode(image,kernal, iterations=1)
    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernal)
    image = cv2.medianBlur(image, 3)
    return(image)

In [ ]:
no_noise = noise_removal(im_bw)
cv2.imwrite("temp/no_noise.png", no_noise)

In [ ]:
display("temp/no_noise.png")

In [ ]:
def thin_front(image):
    import numpy as np
    image = cv2.bitwise_not(image)
    kernal = np.ones((2,2), np.uint8)
    image = cv2.erode(image, kernal, iterations = 1)
    image = cv2.bitwise_not(image)
    return(image)

In [ ]:
eroded_image = thin_front(no_noise)
cv2.imwrite("temp/eroded_image.png", eroded_image)

In [ ]:
display("temp/eroded_image.png")

In [ ]:
def thick_front(image):
    import numpy as np
    image = cv2.bitwise_not(image)
    kernal = np.ones((2,2), np.uint8)
    image = cv2.dilate(image, kernal, iterations = 1)
    image = cv2.bitwise_not(image)
    return(image)

In [ ]:
dilated_image = thick_front(no_noise)
cv2.imwrite("temp/dilated_image.png", dilated_image)

In [ ]:
display("temp/dilated_image.png")

In [ ]:
new = cv2.imread("data/rotatedimage.png")

In [ ]:
import cv2
import os
def getSkewAngle(img) -> float:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (9, 9), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 5))
    dilate = cv2.dilate(thresh, kernel, iterations=2)

    # Find all contours
    contours, hierarchy = cv2.findContours(dilate, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)
    for c in contours:
        rect = cv2.boundingRect(c)
        x,y,w,h = rect
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)

    # Find largest contour and surround in min area box
    largestContour = contours[0]
    print (len(contours))
    minAreaRect = cv2.minAreaRect(largestContour)
    cv2.imwrite("temp/boxes.jpg", img)
    # Determine the angle. Convert it to the value that was originally used to obtain skewed image
    angle = minAreaRect[-1]
    if angle < -45:
        angle = 90 + angle
    return -1.0 * angle



In [ ]:
# Deskew image
def deskew(cvImage):
    angle = getSkewAngle(cvImage)
    return rotateImage(cvImage, -1.0 * angle)

In [ ]:
display("temp/rotated_fixed.jpg")

In [ ]:
display("temp/no_noise.jpg")

In [ ]:
def remove_borders(image):
    contours, heiarchy = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cntsSorted = sorted(contours, key=lambda x:cv2.contourArea(x))
    cnt = cntsSorted[-1]
    x, y, w, h = cv2.boundingRect(cnt)
    crop = image[y:y+h, x:x+w]
    return (crop)

In [ ]:
no_borders = remove_borders(no_noise)
cv2.imwrite("temp/no_borders.jpg", no_borders)
display('temp/no_borders.jpg')

In [ ]:
color = [255, 255, 255]
top, bottom, left, right = [150]*4

In [ ]:
image_with_border = cv2.copyMakeBorder(no_borders, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
cv2.imwrite("temp/image_with_border.jpg", image_with_border)
display("temp/image_with_border.jpg")